In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

# Leer el archivo personas.sav
censo = pd.read_spss("C:/Users/lpescetto/Downloads/Datathon/personas.sav")

# Calcular personas por segmento
personas_por_segmento = censo.assign(ID_segmento=lambda x: x['DPTO'] + x['LOC'] + x['SECC'] + x['SEGM']) \
                              .groupby('ID_segmento').size().reset_index(name='n')

# Leer el archivo base_segmentos_4_.shp
segmentos = gpd.read_file("base_segmentos_4_.shp")

# Ajustar campos en segmentos
segmentos = segmentos.assign(DEPTO=lambda x: x['DEPTO'].str.zfill(2),
                             LOCALIDAD=lambda x: x['LOCALIDAD'].str.zfill(3),
                             SECCION=lambda x: x['SECCION'].str.zfill(2),
                             SEGMENTO=lambda x: x['SEGMENTO'].str.zfill(3),
                             ID_segmento=lambda x: x['DEPTO'] + x['LOCALIDAD'] + x['SECCION'] + x['SEGMENTO']) \
                     .loc[:, ['DEPTO', 'NOMBDEPTO', 'NOMBLOC', 'geometry', 'ID_segmento']]

# Leer el archivo hogares.sav
hogares = pd.read_spss("C:/Users/lpescetto/Downloads/Datathon/hogares.sav")

# Filtrar hogares_nbi
hogares_nbi = hogares.drop(columns=['NBI_MAT', 'NBI_HAC', 'NBI_COC', 'NBI_CALEF', 'NBI_REFRIG', 'NBI_CALENTADOR', 'NBI_CANTIDAD']) \
                     .loc[~hogares.filter(like="NBI").isin([8, 9, 5555]).any(axis=1)] \
                     .assign(NBI_TOTAL=lambda x: x['NBI_VIV'] + x['NBI_AGUA'] + x['NBI_SANEA'] + x['NBI_ELECT'] + x['NBI_CONFORT'] + x['NBI_EDUCACIÓN'],
                             ID_segmento=lambda x: x['DPTO'] + x['LOC'] + x['SECC'] + x['SEGM']) \
                     .groupby('ID_segmento') \
                     .agg(NBI_una=('NBI_TOTAL', lambda x: (x >= 1).sum() / len(x))).reset_index()

# Guardar el archivo segmentos_poblacion_nbi_6.shp
segmentos_poblacion_nbi.to_file("segmentos_poblacion_nbi_6.shp", na_rm=False)

# Leer archivos adicionales
liceo = gpd.read_file("centros ANEP/CENTROS_ANEP_22.shp").query("SUBSISTEMA == 'DGES'")
utu = gpd.read_file("centros ANEP/CENTROS_ANEP_22.shp").query("SUBSISTEMA == 'DGETP'")

# Guardar archivos adicionales
liceo.to_file("liceo.shp")
utu.to_file("utu.shp")

# Leer distancias calculadas con ArcGis
distancias_bomberos = pd.read_csv("Connecting Lines_1.csv")
distancias_policia = pd.read_csv("policía/Connecting Lines_1.csv")
distancias_hospitales = pd.read_csv("hospitales/Connecting Lines_1.csv")
distancias_violencia = pd.read_csv("violencia/Connecting Lines_1.csv")

# Ajustar campos en distancias
for df in [distancias_bomberos, distancias_policia, distancias_hospitales, distancias_violencia]:
    df['ID_segmento'] = df['FindCentroidsOutput..ID_sgmn'].astype(str).str.zfill(10)
    df.rename(columns={'Straight.Line.Distance..Kilometers.': f"distancia_{df.columns[0].lower()}"}, inplace=True)

# Seleccionar columnas relevantes
distancias_bomberos = distancias_bomberos[['ID_segmento', 'distancia_bomberos']]
distancias_policia = distancias_policia[['ID_segmento', 'distancia_policia']]
distancias_hospitales = distancias_hospitales[['ID_segmento', 'distancia_hospitales']]
distancias_violencia = distancias_violencia[['ID_segmento', 'distancia_violencia']]

# Leer archivos con cercanía hechos con ArcGis
escuelas_cercania = pd.read_csv("escuelas_cant.csv")
liceo_cercania = pd.read_csv("liceo_cercania.csv")
utu_cercania = pd.read_csv("utu_cercania.csv")

# Ajustar campos en archivos adicionales
for df in [escuelas_cercania, liceo_cercania, utu_cercania]:
    df['ID_segmento'] = df['ID_sgmn'].astype(str).str.zfill(10)
    df[f"per_por_{df.columns[0].lower()}"] = np.where(df['Count.of.Points'] == 0, 9999, df['n'] / df['Count.of.Points'])
    df.drop(columns=['ID_sgmn', 'Count.of.Points', 'n'], inplace=True)

# Seleccionar columnas relevantes
escuelas_cercania = escuelas_cercania[['ID_segmento', 'per_por_esc']]
liceo_cercania = liceo_cercania[['ID_segmento', 'per_por_lic']]
utu_cercania = utu_cercania[['ID_segmento', 'per_por_utu']]

# Combinar dataframes
segmentos_poblacion_nbi_completo = pd.merge(segmentos_poblacion_nbi, distancias_bomberos, on='ID_segmento', how='left')
segmentos_poblacion_nbi_completo = pd.merge(segmentos_poblacion_nbi_completo, distancias_policia, on='ID_segmento', how='left')
segmentos_poblacion_nbi_completo = pd.merge(segmentos_poblacion_nbi_completo, distancias_hospitales, on='ID_segmento', how='left')
segmentos_poblacion_nbi_completo = pd.merge(segmentos_poblacion_nbi_completo, distancias_violencia, on='ID_segmento', how='left')
segmentos_poblacion_nbi_completo = pd.merge(segmentos_poblacion_nbi_completo, escuelas_cercania, on='ID_segmento', how='left')
segmentos_poblacion_nbi_completo = pd.merge(segmentos_poblacion_nbi_completo, liceo_cercania, on='ID_segmento', how='left')
segmentos_poblacion_nbi_completo = pd.merge(segmentos_poblacion_nbi_completo, utu_cercania, on='ID_segmento', how='left')

# Guardar el archivo segmentos_poblacion_nbi_completo.shp
segmentos_poblacion_nbi_completo.to_file("segmentos_poblacion_nbi_completo.shp", na_rm=False)​